EPITECH PROJECT 2025
TARDIS
tardis_eda
-- Cleans data and retrieve it into cleaned_dataset.csv

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import csv

clean_names = [
    "BORDEAUX ST JEAN",
    "LA ROCHELLE VILLE",
    "PARIS MONTPARNASSE",
    "QUIMPER",
    "TOURS",
    "ST PIERRE DES CORPS",
    "ST MALO",
    "NANTES",
    "PARIS EST",
    "STRASBOURG",
    "DUNKERQUE",
    "LILLE",
    "PARIS VAUGIRARD",
    "RENNES",
    "TOURCOING",
    "CHAMBERY CHALLES LES EAUX",
    "LYON PART DIEU",
    "MONTPELLIER",
    "MULHOUSE VILLE",
    "NICE VILLE",
    "PARIS LYON",
    "BARCELONA",
    "GENEVE",
    "MADRID",
    "BREST",
    "POITIERS",
    "TOULOUSE MATABIAU",
    "MARNE LA VALLEE",
    "MARSEILLE ST CHARLES",
    "FRANCFORT",
    "ANGOULEME",
    "METZ",
    "PARIS NORD",
    "BELLEGARDE (AIN)",
    "MACON LOCHE",
    "PERPIGNAN",
    "DOUAI",
    "VALENCE ALIXAN TGV",
    "LAUSANNE",
    "ANGERS SAINT LAUD",
    "STUTTGART",
    "LAVAL",
    "NANCY",
    "BESANCON FRANCHE COMTE TGV",
    "GRENOBLE",
    "NIMES",
    "SAINT ETIENNE CHATEAUCREUX",
    "ITALIE",
    "ZURICH",
    "VANNES",
    "ANNECY",
    "AVIGNON TGV",
    "MADRID",
    "LE MANS",
    "ST MALO",
    "ARRAS",
    "DIJON VILLE",
    "LE CREUSOT MONTCEAU MONTCHANIN",
    "REIMS",
]

Takes data from dataframe and return the number of cancelled trains by departure station into a graph

In [ ]:
def show_data(df):
    df.columns = df.columns.str.strip()
    df["Departure station"] = (
        df["Departure station"].astype(str).str.upper().str.strip()
    )
    df["Average journey time"] = pd.to_numeric(
        df["Average journey time"], errors="coerce"
    )

    df_grouped = (
        df.groupby("Departure station")["Number of cancelled trains"]
        .mean()
        .reset_index()
    )

    stations = df_grouped["Departure station"].tolist()
    delays = df_grouped["Number of cancelled trains"].tolist()

    plt.figure(figsize=(10, 6))
    plt.plot(stations, delays, marker="o")
    plt.xticks(rotation=45, ha="right")
    plt.title("Retards moyens par station de départ")
    plt.xlabel("Station")
    plt.ylabel("Retard moyen (min)")
    plt.tight_layout()
    plt.grid(True)
    plt.show()

This function returns a dictionary of mean values of all the numeric columns grouped by city

In [ ]:
import numpy as np

def get_data_station(df: pd.DataFrame):
    dic_values = {}
    temp_dic = {}
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    df_grouped = df.groupby("Departure station")[numeric_columns].mean().reset_index()
    to_csv(df_grouped, path="grouped.csv")
    for elt in df_grouped["Departure station"]:
        tab = df_grouped[df_grouped["Departure station"] == elt].iloc[0]
        for i in range(1, len(tab)):
            temp_dic[df_grouped.columns[i]] = tab[i]
        dic_values[elt] = temp_dic
    return dic_values

This group of functions return the same as get_data_station except that the values are stored in list and not they're not getting meaned

In [ ]:
def append_to_dic(line, columns, dic, table_of_dup):
    name = line["Departure station"]
    temp_tab = [line["Date"], line["Departure station"], line["Arrival station"]]
    if name == "None" or temp_tab in table_of_dup:
        return 0
    table_of_dup.append(temp_tab)
    if name not in dic:
        dic[name] = {}
    if "Arrival stations" not in dic[name]:
        dic[name]["Arrival stations"] = []
    else:
        dic[name]["Arrival stations"].append(line["Arrival station"])
    if "Dates" not in dic[name]:
        dic[name]["Dates"] = []
    else:
        dic[name]["Dates"].append(temp_tab[0])
    for elt in columns:
        if elt not in dic[name]:
            dic[name][elt] = []
        dic[name][elt].append(line[elt])
    return 0


def get_data_tab(df: pd.DataFrame):
    dic = {}
    table = []
    df_sorted = df.sort_values("Departure station")
    numeric_columns = df_sorted.select_dtypes(include=[np.number]).columns.tolist()
    for i in range(0, len(df_sorted)):
        append_to_dic(df_sorted.iloc[i], numeric_columns, dic, table)
    return dic

Read the csv given in path:
- With the ; separator
- Encoding in utf-8 to get good values
- Disabled quoting so the quotes are automatically removed
- Espacechar is \
- Skip on bad lines to avoid errors
- Runs for python for better treatment

In [ ]:
def read_csv(filename):
    final_dic = {}
    lignes_corrigees = []

    with open(filename, mode="r", encoding='utf-8', newline='') as f:
        reader = csv.reader(f, delimiter=';', quotechar='"')
        header = next(reader)
        nb_columns = len(header)

        for col in header:
            final_dic[col] = []

        buffer = ""
        for line in f:
            buffer += line
            if buffer.count(";") >= nb_columns - 1:
                try:
                    row = next(csv.reader([buffer], delimiter=";", quotechar='"'))
                    row = [val.replace('\n', ' | ').strip() for val in row]
                    if len(row) == nb_columns:
                        lignes_corrigees.append(row)
                except Exception as e:
                    if not e:
                        print(e)
                    buffer = ""
                buffer = ""

        for row in lignes_corrigees:
            for i, val in enumerate(row):
                final_dic[header[i]].append(val)
    return pd.DataFrame(final_dic)

Write a dataframe to csv with the defaut path as test.csv, can be changed though.
Added separator and disabled index to be exactly like the old csv file

In [ ]:
def to_csv(df: pd.DataFrame, path="test.csv"):
    df.to_csv(path, sep=";", index=None)

Check if delay consistency is good:

For exemple, if there is more trains delayed by 30 minutes than 15 minutes, which is not possible, the row delayed by 30mn is set to row delayed by 15mn

In [ ]:
def fix_delay_consistency(row):
        if row["Number of trains delayed > 30min"] > row["Number of trains delayed > 15min"]:
            row["Number of trains delayed > 30min"] = row["Number of trains delayed > 15min"]
        if row["Number of trains delayed > 60min"] > row["Number of trains delayed > 30min"]:
            row["Number of trains delayed > 60min"] = row["Number of trains delayed > 30min"]
        return row


Is valid date check if the date is on format YYYY-MM (REGEX BLBLBLBLBL)

In [ ]:
def is_valid_date(date_str):
    if not isinstance(date_str, str):
        return False
    if len(date_str) != 7 or date_str[4] != '-':
        return False
    try:
        year = int(date_str[0:4])
        month = int(date_str[5:7])
        return 1 <= month <= 12 and year > 0
    except Exception as e:
        if not e:
            print(e)
        return False

This algorithm to get the closest match works for a list and a name:

- Compare the name with all elements in the correct list
- returns the best match with a distance <=1
- return None if no good matches found

In [ ]:
def hamming_distance(s1, s2):
    if len(s1) != len(s2):
        return np.inf
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))


def get_closest_match(name, correct_list):
    if not name or not isinstance(name, str):
        return "None"

    name = name.strip().upper()
    correct_list_cleaned = [n.strip().upper() for n in correct_list]

    distances = [hamming_distance(name, ref) for ref in correct_list_cleaned]
    min_distance = min(distances)

    if min_distance <= 1:
        best_match_index = distances.index(min_distance)
        return correct_list[best_match_index]
    else:
        return "None"

Clean dataset if a mandatory column is empty

In [ ]:
def missing_mandatory(df, mandatory_cols):
    df_clean = df.copy()
    for col in mandatory_cols:
        if col in df_clean.columns:
            idx_to_drop = df_clean[
                df_clean[col].isna() | (df_clean[col].astype(str).str.lower() == 'nan')
            ].index
            df_clean = df_clean.drop(index=idx_to_drop)
    return df_clean

This function clean data in three steps:
- If the column is a comment or departure/arrival station it'll be cleaned by the function get_closest_match to return the correct name of the station, None for comments
- If it is the date and miswritten, it'll take the line on top
- If it is none of the above, it is certainly a numeric column so if Null value it is replaced by 0

In [ ]:
mandatory = ["Date", "Service", "Departure station", "Arrival station"]

def clean_data(df: pd.DataFrame):
    
    df = missing_mandatory(df, mandatory)
    df = df.drop_duplicates(
        subset=mandatory,
        keep="first"
    )
    # 2. Correction par colonne
    for column in df.columns:
        col_lower = column.lower()

        # Nettoyage des stations + commentaires
        if column in ["Departure station", "Arrival station"] or "comment" in col_lower:
            df[column] = (
                df[column]
                .fillna("")
                .astype(str)
                .str.strip()
                .apply(lambda x: get_closest_match(x, clean_names))
            )

        # Dates au format YYYY-MM
        elif "date" in col_lower:
            df[column] = df[column].apply(lambda x: x if is_valid_date(x) else pd.NA)
            df[column] = df[column].fillna(method='ffill')

        # Service
        elif column.lower() == "service":
            df[column] = (
                df[column]
                .fillna("")
                .astype(str)
                .str.strip()
                .apply(lambda x: x if x in ["National", "International"] else "International")
            )
        # Colonnes numériques
        else:
            try:
                df[column] = pd.to_numeric(df[column], errors='coerce')

                # Ca dégage toutes les valeurs < 0 à 0 (comptes & délais)
                df[column] = df[column].apply(lambda x: max(x, 0) if pd.notna(x) else 0)

            except Exception as e:
                if not e:
                    print(e)
                df[column] = df[column].fillna("")

    df = df.apply(fix_delay_consistency, axis=1)

    # 4. Supprimer doublons (même départ/arrivée)

    return df


In [ ]:
def get_data_program(df: pd.DataFrame):
    data_values_dic = get_data_station(df)
    data_values_tab = get_data_tab(df)
    return data_values_dic, data_values_tab

In [ ]:
def main():
    # Lecture du fichier csv
    df = read_csv("dataset.csv")

    # Nettoyage du tableau
    cleaned = clean_data(df)

    # Convertir en csv
    to_csv(cleaned, path="cleaned_dataset.csv")

    # Récupération des données
    return 0

In [ ]:
if __name__ == "__main__":
    main()